In [3]:
import sys, os
# sys.path.append('../python')

from util import *

In [4]:
ticker_dict, tickerSymbols = get_categorical_tickers()
start="2010-01-01"
end="2019-12-31"
tickeranalysis = tickerSymbols[0]
tickerSymbol = tickeranalysis
start = start
end = end
depth = 0
naive = True

In [5]:
tickerData = yf.Ticker(tickerSymbol)
tickerDf = yf.download(tickerSymbol, start=start, end=end)
tickerDf = tickerDf['Adj Close']
data = tickerDf
y = data.values
y

[*********************100%***********************]  1 of 1 completed


array([ 99600.,  99710.,  99850., ..., 339650., 338920., 338750.])

In [8]:
past_history = 60
forward_look = 1

training_size = int(y.size * 0.8)
# print(training_size)

training_mean = y[:training_size].mean()
# print(training_mean)  # get the average

training_std = y[:training_size].std()
# print(training_std)  # std = a measure of how far away individual measurements tend to be from the mean value of a data set.

y = (y - training_mean) / training_std  # prep data, use mean and standard deviation to maintain distribution and ratios
# print(y)

xtrain, ytrain = data_preprocess(y, 0, training_size, past_history, forward_look)

xtest, ytest = data_preprocess(y, training_size, None, past_history, forward_look)

In [7]:
def data_preprocess(dataset, iStart, iEnd, sHistory, forward_look):
        data = []
        target = []
        iStart += sHistory
        if iEnd is None:
            iEnd = len(dataset) - forward_look + 1
        for i in range(iStart, iEnd):
            indices = range(i - sHistory, i)  # set the order
            if forward_look > 1:
                fwd_ind = range(i, i + forward_look)
                fwd_entity = np.asarray([])
                fwd_entity = np.append(fwd_entity, dataset[fwd_ind])
            reshape_entity = np.asarray([])
            reshape_entity = np.append(reshape_entity, dataset[
                indices])  # Comment this out if there are multiple identifiers in the feature vector
            data.append(np.reshape(reshape_entity, (sHistory, 1)))  #
            if forward_look > 1:
                target.append(np.reshape(fwd_entity, (forward_look, 1)))
            else:
                target.append(dataset[i])
        data = np.array(data)
        target = np.array(target)
        return data, target

In [9]:
def custom_loss_def(y_true,y_pred):
        weights = np.float32(1.0 + 1.0*np.linspace(0,forward_look-1,forward_look)/200.0)
        return tf.math.reduce_mean(tf.math.square(tf.multiply(weights,y_true - y_pred)))

In [10]:
def create_p_test_train():
        BATCH_SIZE = 30
        BUFFER_SIZE = y.size
        p_train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
        p_train = p_train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).repeat()
        x = p_train
        p_test = tf.data.Dataset.from_tensor_slices((xtest, ytest))
        p_test = p_test.batch(BATCH_SIZE).repeat()
        return p_train, p_test


In [11]:
p_train, p_test = create_p_test_train()

In [21]:
custom_loss = False
def model_LSTM():
    
        model = tf.keras.models.Sequential()
        if naive:
            model.add(tf.keras.layers.LSTM(20, input_shape = xtrain.shape[-2:]))
        else:
            model.add(tf.keras.layers.LSTM(20, return_sequences=True, input_shape = xtrain.shape[-2:]))
        for i in range(depth):
            model.add(tf.keras.layers.LSTM(20, return_sequences=True))
        if naive is False:
            model.add(tf.keras.layers.LSTM(20))
        model.add(tf.keras.layers.Dense(forward_look))
        if custom_loss:
            model.compile(optimizer='Adam',
                      loss=custom_loss_def, metrics=['mse','mae'])
        else:
            model.compile(optimizer='Adam',
                               loss='mse', metrics=['mse','mae'])

        create_p_test_train()
        hist = model.fit(p_train, epochs = 50, steps_per_epoch = 200,
                  validation_data = p_test, validation_steps = 50,
                  verbose = 0)
        xt = xtest
        yt = ytest
        ts = tickerSymbol
        
        infer_values(xt, yt, ts)

In [19]:
infer_train = True
values = 200
# print(xtest[1,:,:])
# print(xtest.shape[2])

In [27]:
ytest[:values-1,0,0]

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [20]:
 def infer_values(xtest, ytest, ts):
        pred = []
        pred_update = []
        usetest = xtest.copy()
        if infer_train:
            pred_train = []
            pred_update_train = []
            usetest_train = xtrain.copy()
        for i in range(values):
            y_pred = model.predict(xtest[i,:,:].reshape(1,xtest.shape[1],xtest.shape[2]))[0][:]
#             print(y_pred)
            y_pred_update = model.predict(usetest[i,:,:].reshape(1,xtest.shape[1],xtest.shape[2]))[0][:]
            pred.append(y_pred)
            pred_update.append(y_pred_update)
            usetest[np.linspace(i+1,i+past_history-1,past_history-1,dtype=int),np.linspace(past_history-2,0,past_history-1,dtype=int),:] =  y_pred_update[0]
            if infer_train:
                y_pred_train = model.predict(xtrain[i, :, :].reshape(1, xtrain.shape[1], xtrain.shape[2]))[0][:]
                y_pred_update_train = \
                model.predict(usetest_train[i, :, :].reshape(1, xtrain.shape[1], xtrain.shape[2]))[0][:]
                pred_train.append(y_pred_train)
                pred_update_train.append(y_pred_update_train)
                usetest_train[np.linspace(i + 1, i + past_history - 1, past_history - 1, dtype=int),
                np.linspace(past_history - 2, 0, past_history - 1, dtype=int), :] = y_pred_update_train[0]
        pred = np.array(pred)
#         print(pred)
        pred_update = np.array(pred_update)
        RMS_error = hist.history['val_mse'][-1]
        RMS_error_train = hist.history['mse'][-1]
        MAE_error = hist.history['val_mae'][-1]
        MAE_error_train = hist.history['mae'][-1]
        if infer_train:
            pred = np.array(pred)
            pred_update_train = np.array(pred_update_train)
        if forward_look>1:
            RMS_error_update = (np.mean(((ytest[:values - 1, 0, 0] - pred_update[1:, 0]) / (
            ytest[:values - 1, 0, 0])) ** 2)) ** 0.5/batch_size
            if infer_train:
                RMS_error_update_train = (np.mean(((ytrain[:values - 1, 0, 0] - pred_update_train[1:, 0]) / (
                    ytrain[:values - 1, 0, 0])) ** 2)) ** 0.5/batch_size
        else:
            RMS_error_update = (np.mean(
                ((ytest[:values - 1] - pred_update[1:]) / (ytest[:values - 1])) ** 2)) ** 0.5/batch_size
            if infer_train:
                RMS_error_update_train = (np.mean(((ytrain[:values - 1] - pred_update_train[1:]) / (
                    ytrain[:values - 1])) ** 2)) ** 0.5/batch_size


In [16]:
def arch_plot():
        '''
        Plot the network architecture
        '''
        dot_img_file = '../images/LSTM_arch_depth%d_naive%d.png' %(depth, int(naive))
        tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)
    

In [ ]:
def plot_test_values(self):
        '''
        Plot predicted values against actual values
        '''
        plt.figure()
        if self.forward_look>1:
            plt.plot(self.yt[:self.values-1,0,0],label='actual (%s)'%self.ts)
            plt.plot(self.pred[1:,0],label='predicted (%s)'%self.ts)
            plt.plot(self.pred_update[1:,0],label='predicted (update)')
            plt.xlabel("Days")
            plt.ylabel("Normalized stock price")
            plt.title('The relative RMS error is %f' % self.RMS_error)
            plt.legend()
            plt.savefig('../images/Stock_prediction_%d_%d_%d_%d_%s_%s.png' % (
            self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)))
            plt.clf()
            plt.figure()
            plt.plot(self.pred[1:, 0]-self.pred_update[1:,0], label='difference (%s)' % self.ts)
            plt.xlabel("Days")
            plt.ylabel("Prediction difference")
            plt.savefig('../images/Difference_%d_%d_%d_%d_%s_%s.png' % (
            self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)))
            plt.clf()
            np.savez('../save_mat/store_%d_%d_%d_%d_%s_%s.png'%(
            self.depth,int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)),
            y = self.yt[:self.values-1,0,0],pred = self.pred[1:,0],pred_up = self.pred_update[1:,0])
        else:
            plt.plot(self.yt[:self.values-1],label='actual (%s)'%self.ts)
            plt.plot(self.pred[1:],label='predicted (%s)'%self.ts)
            plt.plot(self.pred_update[1:],label='predicted (update)')
            plt.xlabel("Days")
            plt.ylabel("Normalized stock price")
            plt.title('The relative RMS error is %f' % self.RMS_error)
            plt.legend()
            plt.savefig('../images/Stock_prediction_%d_%d_%d_%d_%s_%s.png'%(
            self.depth,int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)))
            plt.clf()
            plt.figure()
            plt.plot(self.pred[1:] - self.pred_update[1:], label='difference (%s)' % self.ts)
            plt.xlabel("Days")
            plt.ylabel("Prediction difference")
            plt.savefig('../images/Difference_%d_%d_%d_%d_%s_%s.png' % (
            self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)))
            plt.clf()
            np.savez('../save_mat/store_%d_%d_%d_%d_%s_%s.png' % (
            self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)),
            y=self.yt[:self.values - 1], pred=self.pred[1:], pred_up=self.pred_update[1:])
        print('The relative test RMS error is %f'%self.RMS_error)
        print('The relative test MAE is %f' % self.MAE_error)
        print('The relative test RMS error for the updated dataset is %f' % self.RMS_error_update)
        if self.infer_train:
            print('The relative train RMS error is %f' % self.RMS_error_train)
            print('The relative train MAE is %f' % self.MAE_error_train)
            print('The relative train RMS error for the updated dataset is %f' % self.RMS_error_update_train)
  

In [ ]:
def plot_bot_decision(self):
        '''
        calculate investment growth from the inferred prediction value and plot the resulting growth
        '''
        if self.forward_look > 1:
            ideal = self.yt[:self.values - 1, 0, 0]
            pred = np.asarray(self.pred[1:, 0]).reshape(-1,)
            pred_update = np.asarray(self.pred_update[1:, 0]).reshape(-1,)
        else:
            ideal = self.yt[:self.values - 1]
            pred = np.asarray(self.pred[1:]).reshape(-1,)
            pred_update = np.asarray(self.pred_update[1:]).reshape(-1,)
        control_ideal = get_control_vector(ideal)
        control_pred = get_control_vector(pred)
        control_pred_update = get_control_vector(pred_update)
        bot_ideal = buy_and_sell_bot(ideal, control_ideal)
        bot_pred = buy_and_sell_bot(ideal, control_pred)
        bot_pred_update = buy_and_sell_bot(ideal, control_pred_update)
        plt.figure()
        plt.plot(bot_ideal, label='Ideal case (%.2f)'%bot_ideal[-1])
        plt.plot(bot_pred, label='From prediction (%.2f)'%bot_pred[-1])
        plt.plot(bot_pred_update, label='From prediction (updated) (%.2f)'%bot_pred_update[-1])
        plt.plot(ideal / ideal[0] * 100.0, label='Stock value(%s)' % self.ts)
        plt.xlabel("Days")
        plt.ylabel("Percentage growth")
        plt.legend()
        plt.savefig('../images/Bot_prediction_%d_%d_%d_%d_%s_%s.png' % (self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)))
        np.savez('../save_mat/bot_%d_%d_%d_%d_%s_%s.png' % (
        self.depth, int(self.naive), self.past_history, self.forward_look, self.ts, int(self.custom_loss)),
        ideal = bot_ideal, pred  = bot_pred, pred_up = bot_pred_update)
        plt.clf()
